In [ ]:
#!pip install sklearn

In [ ]:
import pandas as pd
import math

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from warnings import simplefilter

pd.options.mode.chained_assignment = None  # default='warn'import pandas as pd
pd.set_option('display.precision', 5)

In [57]:
df = pd.read_csv('train.csv')
df

,id,date,year_week,product_number,reporterhq_id,prod_category,specs,display_size,segment,sales_units,inventory_units
0,202119-6909,2021-05-15,202119,6909,93,Arale,21274,13.3,Premium,2.0,35.0
1,202120-6909,2021-05-22,202120,6909,93,Arale,21274,13.3,Premium,0.0,70.0
2,202121-6909,2021-05-29,202121,6909,93,Arale,21274,13.3,Premium,3.0,137.0
3,202122-6909,2021-06-05,202122,6909,93,Arale,21274,13.3,Premium,0.0,274.0
4,202123-6909,2021-06-12,202123,6909,93,Arale,21274,13.3,Premium,0.0,333.0
...,...,...,...,...,...,...,...,...,...,...,...
15449,202317-247737,2023-04-29,202317,247737,15,Arale,10637,13.3,Premium,0.0,62.0
15450,202317-247737,2023-04-29,202317,247737,24,Arale,10637,13.3,Premium,0.0,3.0
15451,202318-247737,2023-05-06,202318,247737,3,Arale,10637,13.3,Premium,22.0,0.0
15452,202318-247737,2023-05-06,202318,247737,15,Arale,10637,13.3,Premium,68.0,78.0


In [58]:
grouped = df.groupby('product_number') #grouped['product_number']
grouped_list = [grouped.get_group(x) for x in grouped.groups]

In [74]:
def fill_gaps(data):
  sale_inv = list(zip(data['sales_units'],data['inventory_units']))
  for i in range(len(sale_inv)):

    last_inv = 0
    next_inv = 0
    (act_sales, act_inv) = sale_inv[i]
    if i > 0: 
      (last_sales, last_inv) = sale_inv[i-1]
    if i < len(sale_inv)-1:     
      (next_sales, next_inv) = sale_inv[i+1]


    if math.isnan(act_sales) and math.isnan(act_inv):
      act_inv = (last_inv+next_inv)/2
    if math.isnan(act_inv):
      act_inv = last_inv - act_sales
      sale_inv[i] = (act_sales, act_inv)
    elif math.isnan(act_sales):
      act_sales = last_inv - act_inv
      sale_inv[i] = (act_sales, act_inv)


    data.drop('sales_units', inplace=True,axis=1)
    data.drop('inventory_units', inplace=True,axis=1)

    new_data = pd.DataFrame(sale_inv,columns=['sales_units','inventory_units'])
    data = pd.concat([data,new_data],axis=1)

  return data

In [75]:
for prod in grouped_list:

    # Data preparation
    # ==============================================================================

    # Format the date
    prod['date'] = pd.to_datetime(prod['date'], format='%Y-%m-%d')
    # Convert year_week into weeks 0-max
    prod['year'] = round(prod['year_week']/100)
    first_year = min(prod['year'])
    prod['week'] = ((prod['year_week']/100)%1)*100
    prod['week'] = prod['week'] + (52*(prod['year']-first_year))

    # prod.drop('specs', axis=1, inplace=True)
    # prod.drop('segment', axis=1, inplace=True)
    # prod.drop('display_size', axis=1, inplace=True)
    
    prod = fill_gaps(prod)

      # Checking
    sale_inv = list(zip(prod['sales_units'],prod['inventory_units']))
    non_sales = 0
    non_inv = 0
    non_non = 0
    
    for i in range(len(sale_inv)):
        (sales, inv) = sale_inv[i]
        if math.isnan(sales) and math.isnan(inv):
            non_non += 1
        elif math.isnan(inv):
            non_inv += 1
        elif math.isnan(sales):
            non_sales += 1


    print("NaN Nothing: " + str(non_non))
    print("NaN inventory: " + str(non_inv))
    print("NaN sales: " + str(non_sales))
    

    print(prod.columns)
    

KeyError: 'sales_units'